In [ ]:
#!python -m spacy download en

In [1]:
import doc_helper
from spacy_helper import Spacy_helper
import codecs
import spacy
import os

In [2]:
import ast
import sys

def look_into_module(module_name):
    print('module: '+module_name)
    with open(module_name, "rt") as file:
        tree = ast.parse(file.read(), filename=module_name)
    for func in (f for f in tree.body if isinstance(f, ast.FunctionDef)):
        print(' func: '+func.name)
        
    for clazz in (f for f in tree.body if isinstance(f,ast.ClassDef)):
        print(' class: '+clazz.name)
    print ''
    
look_into_module('doc_helper.py')
look_into_module('spacy_helper.py')

# print dir(doc_helper)
# print dir(Spacy_helper)

module: doc_helper.py
 func: make_doc_one_liner
 func: add_file_descriptor
 func: read_text

module: spacy_helper.py
 class: Spacy_helper



In [3]:
# %load doc_helper.py
import codecs

def make_doc_one_liner(doc_file):
    """
    turns a document with one sentence per line into a document with all sentences in one line
    returns the path of the new file
    """
    with codecs.open(doc_file, 'r', 'UTF-8') as file_int:
        new_file_path = add_file_descriptor(doc_file,'1Line')
        with codecs.open(new_file_path, 'w', 'UTF-8') as file_out:
            for l in file_int:
                file_out.write(l.strip() +' ')
    return new_file_path

def add_file_descriptor(file_name, descriptor):
    """
    add a file descriptor which attached to the end of the file name with _<descriptor>
    use for something like _valid, _fault, _1Line
    """
    ending = file_name[file_name.rfind('.'):]
    return file_name[:file_name.rfind('.')] + '_' + descriptor + ending


def read_text(doc_file):
    """
    utf-8 read whole document
    """
    with codecs.open(doc_file, 'r', 'UTF-8') as file_int:
        return file_int.read()
    
def stream_file(doc_file):
    with codecs.open(doc_file, 'r', 'UTF-8') as file_int:
        for line in file_int:
            yield line

In [7]:
ia_scifi_v3 =  '../../data/scifi-corpus/internet_archive_scifi_v3.txt'
sci_fi = read_text(ia_scifi_v3)

In [13]:
with codecs.open(ia_scifi_v3, 'r', 'UTF-8') as file_int:
    for line in file_int:
        print line

KeyboardInterrupt: 

In [6]:
# for a in stream_file(ia_scifi_v3):
#     print a

KeyboardInterrupt: 

In [ ]:
len(sci_fi)

In [ ]:
ne_file = '../../data/NAIL_DATAFIELD_txt/adams, thomas-r-a-new-model-for-the-study-of-the-book_valid.txt'

new_file = doc_helper.make_doc_one_liner(ne_file)
print new_file
with codecs.open(new_file, 'r', 'UTF-8') as file_in:
    for in_,l in enumerate(file_in):
        print in_

In [ ]:
spacy_helper = Spacy_helper()
doc_clean_tokenz = list(spacy_helper.doc_clean_tokenz(new_file))

In [ ]:
# print tokenz_clean

doc = doc_clean_tokenz[0]
print 'doc', type(doc), len(doc)
tokenz_clean = doc_clean_tokenz[1]
print 'tokenz_clean',type(tokenz_clean), len(tokenz_clean), type(tokenz_clean[1]), tokenz_clean[1]
clean_lemmas = list(spacy_helper.doc_yield_lemmas(tokenz_clean))
print 'clean_lemmas', type(clean_lemmas),type(clean_lemmas[0]), len(clean_lemmas)

# print clean_lemmas
# doc

next step
read all documents in the folder and create dictionary of clean lemmas


In [ ]:
spacy_helper = Spacy_helper()

In [ ]:
# progress bar

def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [ ]:
folders = ['own_mixed_collection']
# ['digital_and_internet_theory','arts_arthistory_aesthetics','ecology_climate_permaculture_collapse',
#         'law_legal_theory_prison_ip','library_and_archive_theory','political_theory_anarchist']

def clean_lemmas(folder):
    for index,file_ in enumerate(os.listdir(folder)):
        print index
        if '_valid.txt' in file_:
            print file_
            clean_tokenz = list(spacy_helper.doc_clean_tokenz(folder + file_))[1]
            yield spacy_helper.doc_yield_lemmas(clean_tokenz)

def clean_lemmas_fi(file_):
    ### TEST!
        if '_valid.txt' in file_:
            print file_
            clean_tokenz = spacy_helper.doc_clean_tokenz(folder + file_)[1]
            yield spacy_helper.doc_yield_lemmas(clean_tokenz)
            
# from gensim.corpora.dictionary import Dictionary
# for fo in folders:
#     folder ='../../data/NAIL_DATAFIELD_txt/parsed_v3/'+fo+'/'
#     log_progress len ([fo for fo in os.listdir(folder) if '_valid.txt' in fo:])
#     clean_lemmas = list(clean_lemmas(folder))
#     di = Dictionary(clean_lemmas)
#     di.save('dicts/'+fo+'.dict')
#     corpus = [di.doc2bow(text) for text in clean_lemmas]
#     corpora.MmCorpus.serialize('corpo/'+fo+'.mm', corpus)  # store to disk, for later use
#     print di
    

In [ ]:
print 1

In [ ]:
# merge_with

folder = 'dicts/'
print os.listdir(folder)
full_dict = None
for file_ in os.listdir(folder):
    if file_[-5:] == '.dict':
        clean_lemmas
        dict_ = Dictionary()
        dict_ = dict_.load(folder + file_)
#         if not full_dict:
#             full_dict = dict_
#         else:
#             print type(full_dict), type(dict_) 
#             full_dict = full_dict.merge_with(dict_)
# full_dict.save(folder + 'all.dict')

In [ ]:
pr

In [ ]:
from gensim import corpora


In [ ]:
print di.__dict__

hist, bins = np.histogram([di.dfs[w] for w in di], bins=50)
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
plt.bar(center, hist, align='center', width=width)
plt.show()

In [ ]:
corpus = [di.doc2bow(text) for text in os.listdir(folder)]

corpora.MmCorpus.serialize('digital_and_internet_theory.mm', corpus)  # store to disk, for later use


In [ ]:
te = Dictionary(["na ne sii".split(), "ne ne uii".split()])
print te
for a in te:
    print a,te[a]